1- Getting information from "Companies" colection

In [1]:
from pymongo import MongoClient
import pandas as pd
from pandas import json_normalize
import numpy as np
import requests
import json
import os
from dotenv import load_dotenv
from pymongo import GEOSPHERE
import geopandas as gpd

In [2]:
client=MongoClient("localhost:27017")

In [3]:
client

MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True)

In [4]:
db=client.get_database("ironhack")

In [5]:
c= db.get_collection("companies")

In [6]:
c

Collection(Database(MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True), 'ironhack'), 'companies')

In [7]:
#c.find_one()

For this project I have chosen New York City, Moscow and Paris as possible locations of the new office. 

In [8]:
c.distinct("offices")[0]

{'description': None,
 'address1': None,
 'address2': None,
 'zip_code': None,
 'city': None,
 'state_code': None,
 'country_code': 'CAN',
 'latitude': 56.130366,
 'longitude': -106.346771}

## New York

In [9]:
ny = list(c.find({"offices.city":"New York"}))

In [10]:
ny2 = list(c.find({"offices.city":"New York City"}))
#navigating through the collection I had realized that New York is writen in two different ways

In [11]:
nyc1=[]
nyc1.append(ny) 
nyc1.append(ny2)
#we are able to get all the companies in New York by appending the two list

In [12]:
nyc=[]
nyc=[item for l in nyc1 for item in l]

In [13]:
len(nyc)

870

In [14]:
m = list(c.find({"offices.city":"Moscow"})) 

In [15]:
len(m)

28

In [16]:
 p = list(c.find({"offices.city":"Paris"})) 

In [17]:
len(p)

179

In [18]:
nyc_of = []
for i in range(len(nyc)):
    dic = {}
    dic["Name"] = nyc[i]["name"]
    dic['number_of_employees']=nyc[i]['number_of_employees']
    for x in range(len(nyc[i]["offices"])):
        if nyc[i]["offices"][x]["city"] == "New York" or nyc[i]["offices"][x]["city"] == "New York City":
            dic["City"] = nyc[i]["offices"][x]["city"]
            dic["Latitude"] = nyc[i]["offices"][x]["latitude"]
            dic["Longitude"] = nyc[i]["offices"][x]["longitude"]
            dic["Coordinates"] = {"type": "Point", "coordinates": [nyc[i]["offices"][x]["latitude"], nyc[i]["offices"][x]["longitude"]]}   
    nyc_of.append(dic)



In [19]:
nyc_of[0]

{'Name': 'Wetpaint',
 'number_of_employees': 47,
 'City': 'New York',
 'Latitude': 40.7237306,
 'Longitude': -73.9964312,
 'Coordinates': {'type': 'Point', 'coordinates': [40.7237306, -73.9964312]}}

In [21]:
df_ny=pd.DataFrame(nyc_of)
df_ny.head()

,Name,number_of_employees,City,Latitude,Longitude,Coordinates
0,Wetpaint,47.0,New York,40.723731,-73.996431,"{'type': 'Point', 'coordinates': [40.7237306, ..."
1,Facebook,5299.0,New York,40.755716,-73.979247,"{'type': 'Point', 'coordinates': [40.7557162, ..."
2,Joost,0.0,New York,40.746497,-74.009447,"{'type': 'Point', 'coordinates': [40.7464969, ..."
3,Viacom,NaN,New York,40.757725,-73.986011,"{'type': 'Point', 'coordinates': [40.757725, -..."
4,AddThis,120.0,New York,40.724604,-73.996876,"{'type': 'Point', 'coordinates': [40.724604, -..."


## Moscu

In [22]:
mos_of = []
for i in range(len(m)):
    dic2 = {}
    dic2["Name"] = m[i]["name"]
    dic2['number_of_employees']=m[i]['number_of_employees']
    
    for x in range(len(m[i]["offices"])):
        if m[i]["offices"][x]["city"] == "Moscow":
            dic2["City"] = m[i]["offices"][x]["city"]
            dic2["Latitude"] = m[i]["offices"][x]["latitude"]
            dic2["Longitude"] = m[i]["offices"][x]["longitude"]
            dic2["Coordinates"] = {"type": "Point", "coordinates": [m[i]["offices"][x]["latitude"], m[i]["offices"][x]["longitude"]]}   
    mos_of.append(dic2)

In [24]:
df_m=pd.DataFrame(mos_of)
df_m.head()

,Name,number_of_employees,City,Latitude,Longitude,Coordinates
0,Google,28000.0,Moscow,NaN,NaN,"{'type': 'Point', 'coordinates': [None, None]}"
1,AllofMP3,NaN,Moscow,55.755786,37.617633,"{'type': 'Point', 'coordinates': [55.755786, 3..."
2,Enforta,NaN,Moscow,NaN,NaN,"{'type': 'Point', 'coordinates': [None, None]}"
3,Badoo,250.0,Moscow,NaN,NaN,"{'type': 'Point', 'coordinates': [None, None]}"
4,Rutube,80.0,Moscow,52.789450,36.427350,"{'type': 'Point', 'coordinates': [52.78945, 36..."


## Paris

In [25]:
p_of = []
for i in range(len(p)):
    dic3 = {}
    dic3["Name"] = p[i]["name"]
    dic3['number_of_employees']=p[i]['number_of_employees']

    for x in range(len(p[i]["offices"])):
        if p[i]["offices"][x]["city"] == "Paris":
            dic3["City"] = p[i]["offices"][x]["city"]
            dic3["Latitude"] = p[i]["offices"][x]["latitude"]
            dic3["Longitude"] = p[i]["offices"][x]["longitude"]
            dic3["Coordinates"] = {"type": "Point", "coordinates": [p[i]["offices"][x]["latitude"], p[i]["offices"][x]["longitude"]]}   
    p_of.append(dic3)

In [26]:
df_p=pd.DataFrame(p_of)
df_p.head()

,Name,number_of_employees,City,Latitude,Longitude,Coordinates
0,Google,28000.0,Paris,48.869606,2.332749,"{'type': 'Point', 'coordinates': [48.869606, 2..."
1,Zlio,NaN,Paris,48.862825,2.381836,"{'type': 'Point', 'coordinates': [48.862825, 2..."
2,Dailymotion,120.0,Paris,NaN,NaN,"{'type': 'Point', 'coordinates': [None, None]}"
3,Netvibes,35.0,Paris,48.870806,2.346680,"{'type': 'Point', 'coordinates': [48.870806, 2..."
4,TVtrip,NaN,Paris,48.856667,2.350987,"{'type': 'Point', 'coordinates': [48.856667, 2..."


## Dataframes cleaning

As I need the coordinates to study the placements I clean all the null values from the dataframes

In [28]:
df_ny.dropna(how="any", inplace=True)
df_p.dropna(how="any", inplace=True)
df_m.dropna(how="any", inplace=True)

My company has 87 employees, then I need a place where they could fit: I filter by the companies with a similar number of employees.

In [29]:
todo= pd.concat([df_ny,df_p,df_m])

In [30]:
todo.isna().sum()

Name                   0
number_of_employees    0
City                   0
Latitude               0
Longitude              0
Coordinates            0
dtype: int64

In [31]:
filtrado = todo.loc[todo["number_of_employees"]>87]

In [32]:
final=filtrado.loc[filtrado["number_of_employees"]<=250]

In [33]:
final.shape

(62, 6)

In [34]:
points=final.sample(n=3,weights='number_of_employees')

The enterprises selected ramdomly from the clean data frames are Solairedirect (Paris), Buka (Moscow), and Networked Insights(New York)

In [39]:
coor_Paris=final.loc[final["Name"]=="Solairedirect"] #isolation of the selected enterprises coordinates

In [40]:
coor_Mos=final.loc[final["Name"]=="BUKA"]
coor_Mos

,Name,number_of_employees,City,Latitude,Longitude,Coordinates
11,BUKA,250.0,Moscow,55.755786,37.617633,"{'type': 'Point', 'coordinates': [55.755786, 3..."


In [74]:
coor_NY=final.loc[final["Name"]=="Networked Insights"]
coor_NY

,Name,number_of_employees,City,Latitude,Longitude,Coordinates
186,Networked Insights,130.0,New York,40.745559,-73.986243,"{'type': 'Point', 'coordinates': [40.745559, -..."


# Foursquare API

The following are the requirements of the staff for the new place:
- Designers like to go to design talks and share knowledge. There must be some nearby companies that also do design.
- 30% of the company staff have at least 1 child.
- Developers like to be near successful tech startups that have raised at least 1 Million dollars.
- Executives like Starbucks A LOT. Ensure there's a starbucks not too far.
- Account managers need to travel a lot.
- Everyone in the company is between 25 and 40, give them some place to go party.
- The CEO is vegan.
- If you want to make the maintenance guy happy, a basketball stadium must be around 10 Km.
- The office dog—"Dobby" needs a hairdresser every month. Ensure there's one not too far away.

For being able to evaluate the optimal location, the requirements are given values according to their importance.

In [42]:
design_comp=10
kinder=45
starbucks=20
station=30
bars=50
vegan=20
basket=10
dobby=25
"basketball"

'basketball'

In [43]:
load_dotenv()

True

In [44]:
client_id= os.getenv("client")
secret= os.getenv("secret")

In [45]:
pd.Series(coor_Paris["Latitude"]).values[0]

48.8745951

In [46]:
p_coor=f"{pd.Series(coor_Paris['Latitude']).values[0]}, {pd.Series(coor_Paris['Longitude']).values[0]}"
p_coor

'48.8745951, 2.3210737'

In [47]:
m_coor=f"{pd.Series(coor_Mos['Latitude']).values[0]}, {pd.Series(coor_Mos['Longitude']).values[0]}"
m_coor

'55.755786, 37.617633'

In [48]:
ny_coor=f"{pd.Series(coor_NY['Latitude']).values[0]}, {pd.Series(coor_NY['Longitude']).values[0]}"
ny_coor

'40.745559, -73.986243'

In [49]:
url_fs = 'https://api.foursquare.com/v2/venues/search'

## Paris

In [50]:
lista=["web design company","kindergarten","starbucks","bars","train station","vegan restaurant","veterinary"]

In [51]:
listab=["web design company","kindergarten","starbucks","bars","train station","vegan restaurant","veterinary","basketball court"]

In [52]:
dic_paris={}
for i in lista:
    
    parametros = {
    "client_id": client_id,
    "client_secret": secret,
    "radius":2000,
    "v": "20211115",
    "limit": 200,
    "ll": p_coor,
    "query": i}
    x = requests.get(url= url_fs, params = parametros).json()
    dic_paris[i]=x["response"]["venues"]
    
    

In [53]:
parametros = {"client_id": client_id,
    "client_secret": secret,
    "radius":10000,
    "v": "20211115",
    "limit": 200,
    "ll": p_coor,
    "query": i}
p = requests.get(url= url_fs, params = parametros).json()
dic_paris["basketball court"]=p["response"]["venues"]

In [54]:
p_all=[]
for a in listab:
    for i in range(len(dic_paris[a])):
        dicny={}
        dicny["city"]="Paris"
        dicny["category"]=a
        dicny["Name"]=dic_paris[a][i]["name"]
        dicny["lat"]=dic_paris[a][i]["location"]["lat"]
        dicny["lng"]=dic_paris[a][i]["location"]["lng"]
        dicny["points"]={"type": "Point", "coordinates": [dic_paris[a][i]["location"]["lat"],dic_paris[a][i]["location"]["lng"]]}
        dicny["points-lng-lat"]={"type": "Point", "coordinates": [dic_paris[a][i]["location"]["lng"],dic_paris[a][i]["location"]["lat"]]}
        p_all.append(dicny)

In [55]:
paris=pd.DataFrame(p_all)
paris = gpd.GeoDataFrame(paris, geometry=gpd.points_from_xy(paris.lng, paris.lat))
paris.to_csv("data/paris.csv")

## Moscow

In [56]:
dic_moscow={}
for i in lista:
    
    parametros = {
    "client_id": client_id,
    "client_secret": secret,
    "radius":2000,
    "v": "20211115",
    "limit": 200,
    "ll": m_coor,
    "query": i}
    x = requests.get(url= url_fs, params = parametros).json()
    dic_moscow[i]= x["response"]["venues"]
    

In [57]:
parametros = {"client_id": client_id,
    "client_secret": secret,
    "radius":10000,
    "v": "20211115",
    "limit": 200,
    "ll": m_coor,
    "query": 'basketball court'}
m = requests.get(url= url_fs, params = parametros).json()
dic_moscow["basketball court"]=m["response"]["venues"]

In [58]:
mk_all=[]
for a in listab:
    for i in range(len(dic_moscow[a])):
        dicny={}
        dicny["city"]="Moskow"
        dicny["category"]=a
        dicny["Name"]=dic_moscow[a][i]["name"]
        dicny["lat"]=dic_moscow[a][i]["location"]["lat"]
        dicny["lng"]=dic_moscow[a][i]["location"]["lng"]
        dicny["points"]={"type": "Point", "coordinates": [dic_moscow[a][i]["location"]["lat"],dic_moscow[a][i]["location"]["lng"]]}
        dicny["points-lng-lat"]={"type": "Point", "coordinates": [dic_moscow[a][i]["location"]["lng"],dic_moscow[a][i]["location"]["lat"]]}
        mk_all.append(dicny)

In [59]:
msk_all=pd.DataFrame(mk_all)
msk_all = gpd.GeoDataFrame(msk_all, geometry=gpd.points_from_xy(msk_all.lng, msk_all.lat))

In [60]:
msk_all.to_csv("data/moscu.csv")

## New York

In [61]:
dic_ny={}
for i in lista:
    
    parametros = {
    "client_id": client_id,
    "client_secret": secret,
    "radius":2000,
    "v": "20211115",
    "limit": 200,
    "ll": ny_coor,
    "query": i}
    x = requests.get(url= url_fs, params = parametros).json()
    dic_ny[i]=x["response"]["venues"]
    

In [62]:
parametros = {"client_id": client_id,
    "client_secret": secret,
    "radius":10000,
    "v": "20211115",
    "limit": 200,
    "ll": ny_coor,
    "query": "basketball court"}
x = requests.get(url= url_fs, params = parametros).json()
dic_ny["basketball court"]=x["response"]["venues"]

In [63]:
pd_ny=pd.DataFrame.from_dict(dic_ny, orient='index')


In [64]:
ny2=pd_ny.transpose()
ny2.notnull().sum()

web design company    27
kindergarten           0
starbucks             50
bars                  24
train station         50
vegan restaurant      50
veterinary            22
basketball court      50
dtype: int64

In [65]:
ny_b=[]
for a in listab:
    for i in range(len(dic_ny[a])):
        dicny={}
        dicny["city"]="New York"
        dicny["category"]=a
        dicny["Name"]=dic_ny[a][i]["name"]
        dicny["lat"]=dic_ny[a][i]["location"]["lat"]
        dicny["lng"]=dic_ny[a][i]["location"]["lng"]
        dicny["points"]={"type": "Point", "coordinates": [dic_ny[a][i]["location"]["lat"],dic_ny[a][i]["location"]["lng"]]}
        dicny["points-lng-lat"]={"type": "Point", "coordinates": [dic_ny[a][i]["location"]["lng"],dic_ny[a][i]["location"]["lat"]]}
        ny_b.append(dicny)

In [66]:
ny6=pd.DataFrame(ny_b)
ny6 = gpd.GeoDataFrame(ny6, geometry=gpd.points_from_xy(ny6.lng, ny6.lat))

In [67]:
ny6.to_csv("data/newyork.csv")

# Geoqueries

In [68]:
client = MongoClient("localhost:27017")
db = client.get_database("ironhack")
db

Database(MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True), 'ironhack')

In [69]:
moscu = db.get_collection("moscu")
ny = db.get_collection("newyork")
paris = db.get_collection("paris")

In [73]:
moscu.geometry.create_index([("geometry", GEOSPHERE)])


'geometry_2dsphere'

In [75]:
m_coor

'55.755786, 37.617633'

In [76]:
listab

['web design company',
 'kindergarten',
 'starbucks',
 'bars',
 'train station',
 'vegan restaurant',
 'veterinary',
 'basketball court']

In [83]:
moscu.geometry.aggregate([
   {
     '$near': {
        near: { type: "Point", coordinates: [m_coor] },
        key: "points",
        distanceField: "dist.calculated",
        query: { "category": "starbucks" }}}])

NameError: name 'near' is not defined

In [93]:
query={'near':[m_coor], 'distanceField': "dist.calculated",'query': { "category": "starbucks" }}

In [ ]:
moscu[0]["location"]["coordinates"][::-1]

In [104]:
moscu2=pd.DataFrame(moscu)

TypeError: 'Collection' object is not iterable

In [102]:
total = [] # lista para resultado final
for i in libres_cerca: #iteramos por la lista entera de librerías de antes 
    point = type_point([libreria["longitud"],libreria["latitud"]]) #preparamos tipo point de cada una 
    query = {"geometry": {"$geoIntersects": {"$geometry": point}}} #hacemos la query para buscar el barrio 
    resultado = dis.find_one(query) # hacemos la query 
    libre = {libreria["nombre"]:resultado["properties"]["nombre"]} # nombre de la librería y nombre del distrito de la query
    total.append(libre)

TypeError: pipeline must be a list

In [86]:
[m_coor]

['55.755786, 37.617633']